# Document Loaders trong LangChain

## Giới thiệu

Document Loaders là thành phần quan trọng trong LangChain, giúp:
- **Tải dữ liệu** từ nhiều nguồn khác nhau (PDF, Web, CSV, Text, ...)
- **Chuẩn hóa** dữ liệu thành format Document chuẩn
- **Trích xuất metadata** hữu ích cho việc xử lý sau này
- **Tích hợp** dễ dàng với các components khác (splitters, embeddings, vector stores)

Trong notebook này, chúng ta sẽ học cách sử dụng các loaders phổ biến nhất.

## Setup môi trường và chuẩn bị dữ liệu mẫu

In [ ]:
# Import các thư viện cần thiết
import os
import requests
from pathlib import Path

# Document Loaders
from langchain_community.document_loaders import (
    TextLoader,
    PyPDFLoader,
    CSVLoader,
    WebBaseLoader,
    UnstructuredMarkdownLoader,
    DirectoryLoader,
    JSONLoader
)

# Utilities
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
import json

In [ ]:
# Tạo thư mục cho data samples
data_dir = Path("sample_data")
data_dir.mkdir(exist_ok=True)

# Tạo file text mẫu
sample_text = """Giới thiệu về LangChain

LangChain là một framework mạnh mẽ cho phát triển ứng dụng với Large Language Models (LLMs).
Framework này cung cấp:

1. Chains: Kết nối các components lại với nhau
2. Agents: Tự động quyết định actions dựa trên input
3. Memory: Lưu trữ context giữa các lần tương tác
4. Callbacks: Theo dõi và debug applications

LangChain hỗ trợ nhiều LLM providers như OpenAI, Anthropic, Cohere, và nhiều hơn nữa.
"""

with open(data_dir / "langchain_intro.txt", "w", encoding="utf-8") as f:
    f.write(sample_text)

print("✓ Đã tạo file text mẫu")

In [ ]:
# Tạo file CSV mẫu
csv_data = {
    "product_name": ["iPhone 15", "Samsung S24", "Google Pixel 8", "OnePlus 12"],
    "category": ["Smartphone", "Smartphone", "Smartphone", "Smartphone"],
    "price_vnd": [25000000, 22000000, 18000000, 16000000],
    "features": [
        "A17 Pro chip, Dynamic Island, 48MP camera",
        "Snapdragon 8 Gen 3, 200MP camera, S Pen support",
        "Google Tensor G3, Best Android camera, 7 years update",
        "Snapdragon 8 Gen 3, 100W charging, Hasselblad camera"
    ],
    "rating": [4.8, 4.7, 4.6, 4.5]
}

df = pd.DataFrame(csv_data)
df.to_csv(data_dir / "products.csv", index=False, encoding="utf-8")

print("✓ Đã tạo file CSV mẫu")
print(df.head())

In [ ]:
# Tạo file JSON mẫu
json_data = {
    "company": "TechViet Solutions",
    "founded": 2020,
    "employees": [
        {
            "name": "Nguyễn Văn A",
            "position": "CEO",
            "department": "Management",
            "skills": ["Leadership", "Strategy", "Business Development"]
        },
        {
            "name": "Trần Thị B",
            "position": "CTO",
            "department": "Technology",
            "skills": ["Python", "Cloud Architecture", "AI/ML"]
        },
        {
            "name": "Lê Văn C",
            "position": "Senior Developer",
            "department": "Technology",
            "skills": ["React", "Node.js", "MongoDB"]
        }
    ],
    "products": [
        {"name": "AI Assistant", "type": "SaaS", "users": 10000},
        {"name": "Data Analytics Platform", "type": "Enterprise", "users": 500}
    ]
}

with open(data_dir / "company_data.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=2)

print("✓ Đã tạo file JSON mẫu")

In [ ]:
# Tạo file Markdown mẫu
markdown_content = """# RAG System Architecture

## Overview
Retrieval-Augmented Generation (RAG) là một kỹ thuật kết hợp khả năng truy xuất thông tin với khả năng sinh văn bản của LLMs.

## Components

### 1. Document Loading
- Load documents từ nhiều nguồn
- Support nhiều formats: PDF, DOCX, HTML, etc.

### 2. Text Splitting
- Chia documents thành chunks nhỏ hơn
- Overlap để maintain context

### 3. Embedding
- Convert text chunks thành vectors
- Sử dụng models như OpenAI Embeddings, Sentence Transformers

### 4. Vector Storage
- Lưu trữ embeddings trong vector databases
- Support similarity search

### 5. Retrieval
- Tìm kiếm relevant chunks dựa trên query
- Ranking và filtering results

### 6. Generation
- Sử dụng retrieved context với LLM
- Generate final response

## Best Practices
- Chunk size: 500-1000 tokens
- Overlap: 10-20%
- Top-k retrieval: 3-5 chunks
"""

with open(data_dir / "rag_architecture.md", "w", encoding="utf-8") as f:
    f.write(markdown_content)

print("✓ Đã tạo file Markdown mẫu")

## 1. TextLoader - Loader cơ bản nhất

In [ ]:
# Load text file
text_loader = TextLoader(
    file_path=str(data_dir / "langchain_intro.txt"),
    encoding="utf-8"
)

# Load documents
text_docs = text_loader.load()

print(f"Số lượng documents: {len(text_docs)}")
print(f"\nDocument đầu tiên:")
print(f"- Content preview: {text_docs[0].page_content[:200]}...")
print(f"- Metadata: {text_docs[0].metadata}")

In [ ]:
# TextLoader với custom metadata
class CustomTextLoader(TextLoader):
    """TextLoader với metadata tùy chỉnh"""
    
    def load(self):
        documents = super().load()
        # Thêm custom metadata
        for doc in documents:
            doc.metadata.update({
                "type": "tutorial",
                "language": "vietnamese",
                "topic": "langchain",
                "word_count": len(doc.page_content.split())
            })
        return documents

custom_loader = CustomTextLoader(str(data_dir / "langchain_intro.txt"), encoding="utf-8")
custom_docs = custom_loader.load()

print("Document với custom metadata:")
print(f"Metadata: {custom_docs[0].metadata}")

## 2. CSVLoader - Load dữ liệu từ CSV

In [ ]:
# Basic CSV loading
csv_loader = CSVLoader(
    file_path=str(data_dir / "products.csv"),
    encoding="utf-8"
)

csv_docs = csv_loader.load()

print(f"Số lượng documents từ CSV: {len(csv_docs)}")
print(f"\nDocument đầu tiên:")
print(csv_docs[0].page_content)
print(f"\nMetadata: {csv_docs[0].metadata}")

In [ ]:
# CSVLoader với custom settings
custom_csv_loader = CSVLoader(
    file_path=str(data_dir / "products.csv"),
    encoding="utf-8",
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
    },
    source_column="product_name"  # Sử dụng column này cho metadata source
)

custom_csv_docs = custom_csv_loader.load()

# Format lại content cho dễ đọc
for i, doc in enumerate(custom_csv_docs[:2]):
    print(f"\n=== Product {i+1} ===")
    # Parse content thành dict để display đẹp hơn
    lines = doc.page_content.strip().split('\n')
    for line in lines:
        if ':' in line:
            key, value = line.split(':', 1)
            print(f"  {key.strip()}: {value.strip()}")
    print(f"  Source: {doc.metadata['source']}")

## 3. JSONLoader - Load dữ liệu từ JSON

In [ ]:
# JSONLoader với jq schema để extract specific data
from langchain_community.document_loaders import JSONLoader

# Load toàn bộ JSON
def metadata_func(record: dict, metadata: dict) -> dict:
    """Custom function để tạo metadata"""
    metadata["source"] = "company_database"
    metadata["type"] = "employee_record" if "position" in record else "company_info"
    return metadata

# Loader cho employees
employee_loader = JSONLoader(
    file_path=str(data_dir / "company_data.json"),
    jq_schema=".employees[]",  # Extract each employee
    content_key="name",  # Use name as main content
    metadata_func=metadata_func
)

employee_docs = employee_loader.load()

print(f"Số lượng employee documents: {len(employee_docs)}")
for doc in employee_docs:
    print(f"\nEmployee: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")

In [ ]:
# JSONLoader cho complex extraction
# Extract tất cả thông tin thành text
full_json_loader = JSONLoader(
    file_path=str(data_dir / "company_data.json"),
    jq_schema=".",
    text_content=True  # Convert entire JSON to text
)

full_docs = full_json_loader.load()
print(f"Full JSON as text (first 500 chars):")
print(full_docs[0].page_content[:500] + "...")

## 4. WebBaseLoader - Load từ websites

In [ ]:
# WebBaseLoader cơ bản
from langchain_community.document_loaders import WebBaseLoader
import bs4

# Load một webpage
web_loader = WebBaseLoader(
    web_paths=["https://python.langchain.com/docs/modules/data_connection/document_loaders/"],
    bs_kwargs={"parse_only": bs4.SoupStrainer("div", {"class": "markdown"})}
)

try:
    web_docs = web_loader.load()
    print(f"Loaded {len(web_docs)} documents from web")
    print(f"\nContent preview (first 500 chars):")
    print(web_docs[0].page_content[:500] + "...")
    print(f"\nMetadata: {web_docs[0].metadata}")
except Exception as e:
    print(f"Error loading webpage: {e}")
    print("Tip: Đảm bảo bạn có kết nối internet và URL hợp lệ")

In [ ]:
# WebBaseLoader với multiple URLs
multiple_urls = [
    "https://python.langchain.com/docs/get_started/introduction",
    "https://python.langchain.com/docs/get_started/installation"
]

# Custom parser function
def parse_langchain_docs(soup):
    """Extract main content từ LangChain docs"""
    # Tìm main content area
    main_content = soup.find("main") or soup.find("article") or soup.find("div", {"class": "markdown"})
    if main_content:
        # Remove code blocks để text dễ đọc hơn
        for code in main_content.find_all(["pre", "code"]):
            code.decompose()
        return main_content.get_text(separator="\n", strip=True)
    return soup.get_text(separator="\n", strip=True)

# Loader với custom parser
custom_web_loader = WebBaseLoader(
    web_paths=multiple_urls[:1],  # Load 1 URL để demo
    bs_get_text_kwargs={"separator": "\n", "strip": True}
)

try:
    custom_web_docs = custom_web_loader.load()
    if custom_web_docs:
        print(f"Loaded from {custom_web_docs[0].metadata['source']}")
        print(f"Title: {custom_web_docs[0].metadata.get('title', 'N/A')}")
        print(f"Content words: {len(custom_web_docs[0].page_content.split())}")
except Exception as e:
    print(f"Note: Web loading requires internet connection: {e}")

## 5. PyPDFLoader - Load PDF files

In [ ]:
# Tạo một PDF mẫu (nếu có pypdf)
try:
    from fpdf import FPDF
    
    # Tạo PDF mẫu
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=16)
    pdf.cell(0, 10, txt="LangChain Tutorial", ln=True, align='C')
    pdf.set_font("Arial", size=12)
    pdf.ln(10)
    
    content = [
        "Chapter 1: Introduction to LangChain",
        "",
        "LangChain is a powerful framework for building LLM applications.",
        "It provides modular components that can be combined in various ways.",
        "",
        "Key Features:",
        "- Document loaders for various formats",
        "- Text splitters for chunking",
        "- Vector stores for embeddings",
        "- Chains for complex workflows"
    ]
    
    for line in content:
        pdf.cell(0, 10, txt=line, ln=True)
    
    pdf_path = data_dir / "langchain_tutorial.pdf"
    pdf.output(str(pdf_path))
    print("✓ Đã tạo file PDF mẫu")
    
    # Load PDF
    from langchain_community.document_loaders import PyPDFLoader
    
    pdf_loader = PyPDFLoader(str(pdf_path))
    pdf_docs = pdf_loader.load()
    
    print(f"\nSố pages loaded: {len(pdf_docs)}")
    for i, doc in enumerate(pdf_docs):
        print(f"\nPage {i+1}:")
        print(f"Content: {doc.page_content[:200]}...")
        print(f"Metadata: {doc.metadata}")
        
except ImportError:
    print("Cần install fpdf2 để tạo PDF mẫu: pip install fpdf2")
    print("Và pypdf để sử dụng PyPDFLoader: pip install pypdf")

## 6. UnstructuredMarkdownLoader

In [ ]:
# Load Markdown file
try:
    from langchain_community.document_loaders import UnstructuredMarkdownLoader
    
    md_loader = UnstructuredMarkdownLoader(
        str(data_dir / "rag_architecture.md"),
        mode="elements"  # hoặc "single" cho 1 document
    )
    
    md_docs = md_loader.load()
    
    print(f"Số lượng elements: {len(md_docs)}")
    
    # Show different element types
    element_types = set()
    for doc in md_docs:
        if 'category' in doc.metadata:
            element_types.add(doc.metadata['category'])
    
    print(f"\nElement types found: {element_types}")
    
    # Show some examples
    for i, doc in enumerate(md_docs[:3]):
        print(f"\nElement {i+1}:")
        print(f"Type: {doc.metadata.get('category', 'unknown')}")
        print(f"Content: {doc.page_content[:100]}...")
        
except ImportError:
    print("Cần install unstructured: pip install unstructured")
    
    # Fallback: simple markdown loader
    print("\nSử dụng TextLoader thay thế:")
    md_text_loader = TextLoader(str(data_dir / "rag_architecture.md"), encoding="utf-8")
    md_text_docs = md_text_loader.load()
    print(f"Loaded as text: {len(md_text_docs[0].page_content)} characters")

## 7. DirectoryLoader - Load toàn bộ thư mục

In [ ]:
# Load tất cả text files trong directory
dir_loader = DirectoryLoader(
    str(data_dir),
    glob="**/*.txt",  # Pattern để tìm files
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)

dir_docs = dir_loader.load()

print(f"Tìm thấy {len(dir_docs)} text files")
for doc in dir_docs:
    print(f"\nFile: {doc.metadata['source']}")
    print(f"Preview: {doc.page_content[:100]}...")

In [ ]:
# DirectoryLoader với multiple file types
from langchain_community.document_loaders import TextLoader, CSVLoader

# Tạo thêm vài files để test
with open(data_dir / "notes.txt", "w", encoding="utf-8") as f:
    f.write("Đây là ghi chú về LangChain và RAG systems.")

# Loader cho mixed file types
text_loader_mixed = DirectoryLoader(
    str(data_dir),
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"},
    show_progress=True  # Show loading progress
)

csv_loader_mixed = DirectoryLoader(
    str(data_dir),
    glob="**/*.csv",
    loader_cls=CSVLoader,
    loader_kwargs={"encoding": "utf-8"},
    show_progress=True
)

# Load all documents
all_text_docs = text_loader_mixed.load()
all_csv_docs = csv_loader_mixed.load()

print(f"Total text documents: {len(all_text_docs)}")
print(f"Total CSV documents: {len(all_csv_docs)}")
print(f"\nTất cả files loaded:")

all_docs = all_text_docs + all_csv_docs
for doc in all_docs:
    source = Path(doc.metadata['source']).name
    print(f"- {source}")

## 8. Xử lý Documents sau khi load

In [ ]:
# Text splitting cho documents
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load một document dài
long_text = """LangChain là một framework mạnh mẽ cho việc phát triển ứng dụng AI.

Các thành phần chính của LangChain bao gồm:

1. Models: Tích hợp với các LLM providers
LangChain hỗ trợ nhiều model providers như OpenAI, Anthropic, Cohere, Hugging Face và nhiều hơn nữa.
Mỗi provider có những ưu điểm riêng về hiệu suất, chi phí và tính năng.

2. Prompts: Quản lý và tối ưu hóa prompts
Prompt engineering là một phần quan trọng trong việc làm việc với LLMs.
LangChain cung cấp các tools để tạo, quản lý và tối ưu hóa prompts.

3. Memory: Lưu trữ conversation history
Memory components cho phép lưu trữ và truy xuất thông tin từ các cuộc hội thoại trước.
Điều này rất quan trọng cho việc xây dựng chatbots và các ứng dụng conversational.

4. Chains: Kết nối các components
Chains cho phép kết nối nhiều components lại với nhau để tạo ra workflows phức tạp.
Ví dụ: load document -> split text -> create embeddings -> store in vector database.

5. Agents: Tự động reasoning và action
Agents có thể tự quyết định actions cần thực hiện dựa trên input.
Chúng sử dụng LLMs để reasoning và có thể gọi các tools khác nhau.
"""

# Tạo document
from langchain.schema import Document
long_doc = Document(
    page_content=long_text,
    metadata={"source": "manual", "topic": "langchain_overview"}
)

# Split document
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
    separators=["\n\n", "\n", ".", " ", ""]
)

splits = text_splitter.split_documents([long_doc])

print(f"Original document length: {len(long_doc.page_content)} characters")
print(f"Number of chunks: {len(splits)}")
print("\nChunks:")
for i, chunk in enumerate(splits[:3]):
    print(f"\nChunk {i+1} ({len(chunk.page_content)} chars):")
    print(f"Content: {chunk.page_content}")
    print(f"Metadata: {chunk.metadata}")

## 9. Best Practices và Tips

In [ ]:
# Best Practice 1: Xử lý encoding
def safe_load_text(file_path, encodings=['utf-8', 'latin-1', 'cp1252']):
    """Try multiple encodings để load text file"""
    for encoding in encodings:
        try:
            loader = TextLoader(file_path, encoding=encoding)
            return loader.load()
        except UnicodeDecodeError:
            continue
    raise Exception(f"Could not load file with any encoding: {encodings}")

# Best Practice 2: Add metadata during loading
def enrich_documents(docs, additional_metadata):
    """Thêm metadata cho documents"""
    for doc in docs:
        doc.metadata.update(additional_metadata)
        doc.metadata['char_count'] = len(doc.page_content)
        doc.metadata['word_count'] = len(doc.page_content.split())
        doc.metadata['load_time'] = pd.Timestamp.now().isoformat()
    return docs

# Best Practice 3: Validate loaded content
def validate_documents(docs):
    """Kiểm tra documents sau khi load"""
    issues = []
    
    for i, doc in enumerate(docs):
        # Check empty content
        if not doc.page_content or doc.page_content.isspace():
            issues.append(f"Document {i} has empty content")
        
        # Check metadata
        if 'source' not in doc.metadata:
            issues.append(f"Document {i} missing source metadata")
        
        # Check reasonable size
        if len(doc.page_content) > 1000000:  # 1MB text
            issues.append(f"Document {i} unusually large: {len(doc.page_content)} chars")
    
    return issues

# Test best practices
test_docs = text_loader.load()
enriched_docs = enrich_documents(test_docs, {"project": "langchain_tutorial"})
issues = validate_documents(enriched_docs)

print("Enriched document metadata:")
print(enriched_docs[0].metadata)
print(f"\nValidation issues found: {len(issues)}")
if issues:
    for issue in issues:
        print(f"- {issue}")

## 10. Loader Selection Guide

In [ ]:
# Hàm helper để chọn loader phù hợp
def get_loader_for_file(file_path):
    """Tự động chọn loader dựa trên file extension"""
    file_path = Path(file_path)
    extension = file_path.suffix.lower()
    
    loader_map = {
        '.txt': TextLoader,
        '.csv': CSVLoader,
        '.json': JSONLoader,
        '.md': UnstructuredMarkdownLoader,
        '.pdf': 'PyPDFLoader',  # String vì có thể chưa install
        '.html': 'UnstructuredHTMLLoader',
        '.docx': 'Docx2txtLoader',
    }
    
    loader_class = loader_map.get(extension)
    if not loader_class:
        print(f"No specific loader for {extension}, using TextLoader")
        return TextLoader
    
    if isinstance(loader_class, str):
        print(f"Loader {loader_class} requires additional installation")
        return TextLoader
    
    return loader_class

# Test với các files trong sample_data
print("Loader recommendations:")
for file in data_dir.glob("*"):
    if file.is_file():
        loader = get_loader_for_file(file)
        print(f"- {file.name}: {loader.__name__ if hasattr(loader, '__name__') else loader}")

## Tổng kết

Trong notebook này, chúng ta đã học về Document Loaders:

### 1. **Các loại Loader chính**:
- **TextLoader**: Đơn giản, cho text files
- **CSVLoader**: Structured data từ CSV
- **JSONLoader**: Flexible với jq schema
- **WebBaseLoader**: Scrape web content
- **PyPDFLoader**: Extract từ PDF
- **DirectoryLoader**: Batch loading

### 2. **Key Concepts**:
- Mỗi loader tạo ra `Document` objects
- Documents có `page_content` và `metadata`
- Metadata quan trọng cho filtering và retrieval

### 3. **Best Practices**:
- Handle encoding issues
- Enrich metadata
- Validate loaded content
- Choose appropriate loader
- Split large documents

### 4. **Next Steps**:
- Text Splitters để chunk documents
- Embeddings để convert thành vectors
- Vector Stores để lưu trữ và search

Document Loaders là bước đầu tiên quan trọng trong pipeline RAG!